In [0]:
!pip install pyspark

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d48c73d8-d4ce-4bd9-a12d-0f7858dad4bb/bin/python -m pip install --upgrade pip' command.


In [0]:
!pip install confluent_kafka

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-d48c73d8-d4ce-4bd9-a12d-0f7858dad4bb/bin/python -m pip install --upgrade pip' command.


In [0]:
from confluent_kafka import Consumer, KafkaError

consumer_config = {
    'bootstrap.servers': bootstrap_servers,
    'group.id': 'inventory_consumer_group',
    'auto.offset.reset': 'earliest'
}

inventory_consumer = Consumer(consumer_config)
inventory_consumer.subscribe(['inventory_orders'])

def handle_inventory_data(order):
    # Process inventory data here
    print("Processing inventory data:", order)

try:
    while True:
        msg = inventory_consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break
        order = json.loads(msg.value().decode('utf-8'))
        if order['type'] == 'inventory':
            handle_inventory_data(order)

except KeyboardInterrupt:
    pass

finally:
    inventory_consumer.close()


In [0]:
delivery_consumer = Consumer(consumer_config)
delivery_consumer.subscribe(['delivery_orders'])

# Function to handle delivery data
def handle_delivery_data(order):
    print("Processing delivery data:", order)

try:
    while True:
        msg = delivery_consumer.poll(timeout=1.0)
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                print(msg.error())
                break
        order = json.loads(msg.value().decode('utf-8'))
        if order['type'] == 'delivery':
            handle_delivery_data(order)

except KeyboardInterrupt:
    pass

finally:
    delivery_consumer.close()
